# TPC5: A cabine telefónica

Hoje em dia, as cabines telefónicas, popularizadas pela famosa cabine londrina vermelha, caíram em desuso e têm vindo gradualmente a desaparecer. No entanto, podem ainda ser encontradas num ou noutro local.

Neste problema, pretende-se que implemente uma máquina de estados que modele a interacção dum utilizador com um telefone numa cabine pública.

O telefone reage aos seguintes comandos:

  1. **LEVANTAR** - levantar o auscultador, marca o início duma interacção;
  2. **POUSAR** - pousar o auscultador, fim da interacção, deverá ser indicado o montante a ser devolvido;
  3. **MOEDA \<lista de valores>** - inserção de moedas (só deverá aceitar moedas válidas, para valores inválidos deverá ser gerada uma mensagem de erro): `lista de valores = num, num, ..., num`;
  4. **T=numero** - disca o número ( o número deve ter 9 dígitos excepto se for iniciado por "00"); as diferentes chamadas deverão ser tratadas da seguinte maneira:
    * para números iniciados por "601" ou "641" a chamada é "_bloqueada_";
    * para chamadas internacionais (iniciadas por "00") o utilizador tem que ter um saldo igual ou superior a 1,5 euros, caso contrário deverá ser avisado que o saldo é insuficiente e a máquina volta ao estado anterior; a chamada se for realizada tem um custo de 1,5 euros;
    * para chamadas nacionais (iniciadas por "2") o saldo mínimo e custo de chamada é de 25 cêntimos;
    * para chamadas verdes (iniciadas por "800") o custo é 0;
    * para chamadas azuis (iniciadas por "808") o custo é de 10 cêntimos.
  5. **ABORTAR** - interromper a interacção; a máquina devolve as moedas.

Como extra pode ainda detalhar como é que é devolvido o troco: quantas moedas e de que espécie compõem o troco.

A seguir apresenta-se uma possível interacção exemplo.

```
LEVANTAR
maq: "Introduza moedas."
MOEDA 10c, 30c, 50c, 2e.
maq: "30c - moeda inválida; saldo = 2e60c"
T=601181818
maq: "Esse número não é permitido neste telefone. Queira discar novo número!"
T=253604470
maq: "saldo = 2e35c"
POUSAR
maq: "troco=2e35c; Volte sempre!" ou maq: "troco= 1x2e, 1x20c, 1x10c, 1x5c; Volte sempre!"
```

Nota: as linhas iniciadas por "maq:" correspondem às respostas da máquina.

In [ ]:
import json
import re

class Telefone:
    SALDO:float

    def __init__(self):
        self.FLAG = 0
        self.ESTADO = True
        self.MOEDAS = ['1c','2c', '5c', '10c', '20c', '50c', '1e', '2e']
        self.SALDO = 0

    def convertMoeda(self, moeda):
        moeda = moeda.strip()
        match(moeda[-1:]):
            case 'c':
                return (float(moeda[:-1]) / 100)
            case 'e':
                return float (moeda[:-1])
            case _ :
                return 0    

    def convertSaldo(self):
        self.SALDO = round(self.SALDO, 2)
        valor = str(self.SALDO).split('.')
        if len(valor[1]) == 1:
            valor[1] = valor[1] + '0'
        if valor[0] == '0':
            print("saldo = " + valor[1] + 'c')
        else :
            print("saldo = " + valor[0] + 'e' + valor[1] + 'c')


    def moedaValida(self, moeda):
        if moeda in self.MOEDAS:
            return True
        return False

    def moedasValidas(self, moedas):
        notMoeda = []
        for moeda in moedas:
            if not self.moedaValida(moeda):
                notMoeda.append(moeda)
            else:
                self.SALDO += self.convertMoeda(moeda)
        if len(notMoeda) > 0:
            for i in notMoeda:
                print (i, end=' '),
            print("- moeda inválida;", end=' ')
        self.convertSaldo()

    def numeroValido(self, numero):
        if numero.isdigit():
            if numero[0] == '0' and numero[1] == '0':
                return True
            if len(numero) == 9:
                if re.search(r'^\d{6}\d{0}\d{1}', numero) != [] or re.search(r'^\d{6}\d{4}\d{1}', numero) != []:
                    return True
                if numero[0] == '9':
                    return True
                return True
            return True
        return False

    def custoChamada(self, numero):
        if self.numeroValido(numero):
            if numero[0] == '0' and numero[1] == '0':
                a = self.SALDO - 1.5
            elif numero[0] == '2':
                a = self.SALDO - 0.25
            elif numero[0] == '8'and numero[1] == '0' and numero[2] == '0':
                a = self.SALDO
            elif numero[0] == '8'and numero[1] == '0' and numero[2] == '8':
                a = self.SALDO - 0.1
            elif numero[0] == '9':
                a = self.SALDO - 0.35
            if a < 0 :
                print("Saldo insuficiente para efetuar a chamada.")
            else:
                self.SALDO = a
                self.convertSaldo()
        else :
            print ("Esse número não é permitido neste telefone. Queira discar novo número!")

    def atividade(self):
        estado = input()
        if estado == "LEVANTAR":
            self.ESTADO = True
            self.FLAG=1
            print("Introduza moedas.")
        elif estado[:6:] == "MOEDA " :
            if self.FLAG == 0:
                print("Comando inválido.")
                self.ESTADO = False
            else:
                moedas = estado[6:].replace('.','').replace(' ','').split(',')
                self.moedasValidas(moedas)
        elif estado[0]+estado[1] == "T=":
            numero = estado[2:]
            self.custoChamada(numero)
        elif estado == "POUSAR":
            self.convertSaldo()
            self.SALDO = 0
            self.ESTADO = False
        elif estado == "ABORTAR" :
            self.SALDO = 0
            self.ESTADO = False
        else :
            print("Comando inválido.")
            self.ESTADO = False
            
    def main(self):
        while self.ESTADO:
            self.atividade()
            
if __name__ == "__main__":
    t = Telefone()
    try:
        t.main()
    except KeyboardInterrupt:
        print("\nPrograma terminado pelo utilizador.")